Apply what we learned in this Example 3 to download the mRNA microarray expression data from NCBI (Accession number: GSE14333)

http://www.ncbi.nlm.nih.gov/projects/geo/query/acc.cgi?acc=GSE14333.

1.Write an automated code to download the url links for all 290 samples from accession GSE14333.

In [1]:
!pip install selenium
!pip install fake_useragent

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from fake_useragent import UserAgent
# line 8 - 15 is for letting selenium chrome webdriver to run on Colab
ua = UserAgent()
userAgent = ua.chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(options=chrome_options)

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'http://www.ncbi.nlm.nih.gov/projects/geo/query/acc.cgi?acc=GSE14333'
soup = BeautifulSoup(urlopen(url).read(), 'html.parser')
all_a_tags = soup.find_all('a')
hrefs = [a_tag.get('href') for a_tag in all_a_tags]

sample_url=['https://www.ncbi.nlm.nih.gov'+ keep for keep in hrefs if \
keep.startswith('/geo/query/acc.cgi?acc=GSM')]

2.Write an automated code to download the processed microarray data (i.e., VALUE column: log-base-2 values of quantile normalized MAS5.0 calls) from each sample using the links from previous part. The processed microarray array data is the full data table on the page shown after you click on a sample's link and then click on "View full table". There are 54675 probe sets for each sample.

3.Export the processed microarry data for all samples as a single csv file with name "tryityourself3.csv". The number of columns is 290 and the number of rows is equal to the number of probe sets (54,675). Use the id of the samples as column names (GSE358341, GSE358342, ... etc.) and use the names of probe sets (they are the values under ID_REF) as the row names (i.e., index).

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from tqdm import tqdm
genotype_call =[]
data = {}
ua = UserAgent()
i = 341
for sample in tqdm(sample_url):
  values_list = []
  id_refs_list = []
  userAgent = ua.chrome
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument(f'user-agent={userAgent}')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(sample)

  str1 = '//*[@id="ui-ncbiexternallink-1"]/table/tbody/tr/td/table[6]/tbody/tr[3]/'
  str2 = 'td[2]/table/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/input'
  xpath = str1 + str2
  driver.find_element(By.XPATH, xpath).click()

  window_handles = driver.window_handles
  child = window_handles[1]
  driver.switch_to.window(child)

  geno_call = driver.find_element(By.TAG_NAME, 'pre').text

  l =  [line for line in geno_call.split('\n') if 'ID_REF' not in line]

  for line in l[1:]:
    splits = line.split()
    if len(splits) >= 2:
      id_ref = splits[0]
      values_list.append(splits[1:])
      id_refs_list.append(id_ref)

  x = 'GSM358' + str(i)
  data[x] = values_list
  i+=1
  driver.quit()

df = pd.DataFrame(data,index=id_refs_list)
df.to_excel('tryityourself3.xlsx',index_label = 'ID_REF')

100%|██████████| 290/290 [18:00<00:00,  3.73s/it]
